1

In [1]:
c_5 = lambda a: lambda b: lambda c: lambda d: lambda e: a + b + c + d + e

In [2]:
c_5(1)(2)(3)(4)(5)

15

2

In [3]:
from multiprocessing import Pool, cpu_count

def chunk_product(numbers):
    result = 1
    for num in numbers:
        result *= num ** num
    return result


def reduce_mul(lst):
    if not lst:
        return 1
    return lst[0] * reduce_mul(lst[1:])


def factorial_parallel(n, n_chunks=None):
    if n < 2:
        return 1
    
    if n_chunks is None:
        n_chunks = min(cpu_count(), n)
    
    step = n // n_chunks
    chunks = [range(i, min(i + step, n + 1)) for i in range(1, n+1, step)]
    
    with Pool(n_chunks) as pool:
        partial_results = pool.map(chunk_product, chunks)
    
    return reduce_mul(partial_results)


def factorial(n):
    if n < 2:
        return 1
    return n**n * factorial(n-1)

In [80]:
%%time

factorial_parallel(2000)

CPU times: user 9.51 s, sys: 43 ms, total: 9.55 s
Wall time: 14.3 s


ValueError: Exceeds the limit (4300) for integer string conversion; use sys.set_int_max_str_digits() to increase the limit

In [76]:
%%time

factorial(2000)

CPU times: user 1min 16s, sys: 31.1 ms, total: 1min 16s
Wall time: 1min 16s


ValueError: Exceeds the limit (4300) for integer string conversion; use sys.set_int_max_str_digits() to increase the limit

3

In [4]:
def reduce(f, id, s):  
    n = len(s)  
    if n == 0:  
        return id  
    elif n == 1:  
        return s[0]  
    else:  
        return f(reduce(f, id, s[1:]), s[0])


def add(x, y):
    return x + y

def reduce_worker(args):
    f, identity, chunk = args
    return reduce(f, identity, chunk)

def reduce_parallel(f, id, s, n_chunks=None):
    if n_chunks is None:
        n_chunks = min(cpu_count(), len(s))
    step = len(s) // n_chunks
    
    chunks = [(f, id, s[i: i + step]) for i in range(0, len(s), step)]
    
    with Pool(n_chunks) as pool:
        partial_results = pool.map(reduce_worker, chunks)
    
    return reduce(f, id, partial_results)

In [76]:
%%time 

reduce_parallel(add, 0, [i for i in range(2950)], 2)

CPU times: user 1.89 ms, sys: 9.94 ms, total: 11.8 ms
Wall time: 33.6 ms


4349775

In [77]:
%%time

reduce_parallel(add, 0, [i for i in range(2950)], 1)

CPU times: user 5.74 ms, sys: 1.82 ms, total: 7.57 ms
Wall time: 54.6 ms


4349775

4

In [97]:
import numpy as np

def square(lst):
    for i in range(len(lst)):
        lst[i] **= lst[i]

    return lst

def sum_lists(lists):
    final_list = []
    for lst in lists:
        final_list += lst

    return final_list


def square_parallel(lst, n_chunks=None):
    if n_chunks is None:
        n_chunks = min(cpu_count(), len(lst))
    
    step = len(lst) // n_chunks 
    chunks = [lst[i: i + step] for i in range(0, len(lst), step)]

    with Pool(n_chunks) as pool:
        partial_results = pool.map(square, chunks)
    
    return sum_lists(partial_results)

In [103]:
%%time

square_parallel([i for i in range(10000)])

"finished"

CPU times: user 223 ms, sys: 145 ms, total: 368 ms
Wall time: 1.82 s


'finished'

In [108]:
%%time

array = np.array(range(10000))
array ** array

"finished"

CPU times: user 1.88 ms, sys: 5.32 ms, total: 7.2 ms
Wall time: 6.4 ms


'finished'

5

In [109]:
import pandas as pd
from numba import jit



df = pd.read_csv("data/ncr_ride_bookings.csv")
df.shape

(150000, 21)

In [118]:
df["Ride Distance"] = df["Ride Distance"].fillna(0)


In [122]:
def f(x):
    s = 0
    for i in range(int(x ** 3)):
        s += i
    return s


def apply_chunk(args):
    chunk, f = args
    return chunk.apply(f)

def parallel_apply(df, f, n_chunks=None):
    if n_chunks is None:
        n_chunks = cpu_count()
    
    chunks = np.array_split(df, n_chunks)
    
    with Pool(n_chunks) as pool:
        results = pool.map(apply_chunk, [(chunk, f) for chunk in chunks])
    
    return pd.concat(results)


In [124]:
%%time

parallel_apply(df["Ride Distance"], f)

/home/user/Стільниця/python/my projects/uni_vlasov_labs/Uni_labs/.venv/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


CPU times: user 27.8 ms, sys: 80.4 ms, total: 108 ms
Wall time: 29.9 s


0                  0
1              17578
2            3133756
3          775096878
4         6277433176
             ...    
149995    2072617536
149996      46817326
149997       8166861
149998    4459881790
149999      45253341
Name: Ride Distance, Length: 150000, dtype: int64